# Exercise: Geographical Cluster Analysis of Taxi Rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [0]:
import pandas as pd
import numpy as np
import folium


In [2]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: True
Cloning into 'DataScienceSS20'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 450 (delta 1), reused 2 (delta 0), pack-reused 444
Receiving objects: 100% (450/450), 99.60 MiB | 23.52 MiB/s, done.
Resolving deltas: 100% (173/173), done.
Checking out files: 100% (186/186), done.


In [0]:
# we load the data we have saved after wrangling and pre-processing in block I
train=pd.read_csv(path+'/DATA/train_cleaned.csv')

In [0]:
#select only the culumns with the ride coordinates
coordinates = train[ ['pickup_latitude','pickup_longitude','dropoff_latitude' , 'dropoff_longitude' ] ]

## Clustering approach from the lecture
we will be using simple K-Means:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [0]:
from sklearn.cluster import KMeans

In [0]:
#define number of clusters and create instance
clusters=100
myKMeans=KMeans(n_clusters=clusters, n_jobs=-1)#parallelize to all cores

In [7]:
#train model
myKMeans.fit(coordinates.to_numpy()[:100000,:])#use only subset of the data to make it faster

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [0]:
#get cluster centers
centers=myKMeans.cluster_centers_
    

In [0]:
#draw map: green: start, red: end
cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
for i in range(clusters):
    folium.CircleMarker([centers[i,0], centers[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[i,2], centers[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.PolyLine([ [centers[i,0],centers[i,1]] , [centers[i,2],centers[i,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)

In [10]:
cluster_map

## Exercise 1
Write a function ```show_cluster(cluster_number,...)``` that draws the cluster centers and all start and end points of a given cluster in the map.
* use the ```predict()``` method to map all data in ```train_data``` to a cluster center
* use ```folium.CircleMarker``` to draw all members of a given cluster


In [11]:
df = pd.DataFrame(coordinates) 
a = myKMeans.predict(coordinates)  
df['center'] = a
df

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,center
0,40.721319,-73.844311,40.712278,-73.841610,2
1,40.711303,-74.016048,40.782004,-73.979268,36
2,40.761270,-73.982738,40.750562,-73.991242,94
3,40.733143,-73.987130,40.758092,-73.991567,71
4,40.768008,-73.968095,40.783762,-73.956655,87
...,...,...,...,...,...
399995,40.746032,-73.986585,40.724077,-73.990865,55
399996,40.742359,-73.992882,40.762318,-73.972649,56
399997,40.731558,-73.985598,40.728738,-73.987657,21
399998,40.740735,-74.007692,40.722847,-73.988455,85


In [0]:
def show_cluster(cluster_number):
  cluster_map2 = folium.Map(location = [40.730610,-73.935242],zoom_start = 12)

  filterd = df.loc[df['center'] == cluster_number]
  for i in range(len(filterd)):
    point = filterd.iloc[i]
    folium.CircleMarker([point[0], point[1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                      ).add_to(cluster_map2)
    folium.CircleMarker([point[2], point[3]], radius=3,                
                      color="red", 
                      fill_opacity=0.9
                      ).add_to(cluster_map2)
  return cluster_map2

In [13]:
clu=show_cluster(99)
clu

## Exercise 2
Write a function ```cluster_var(cluster_number,...)``` that computes the intra- and extra cluster variance for a given cluster. Apply it to all clusters and compare the results for k=100 and k=10.